In [1]:
from pandas import DataFrame, concat, read_csv, Series
from time import sleep
import re
from nltk.tokenize import word_tokenize
import spacy
import numpy as np
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
from nltk.corpus import stopwords
from nltk import pos_tag
from itertools import combinations
import warnings
warnings.filterwarnings('ignore')
stop_words = set(stopwords.words('english'))
nlp = spacy.load('en_core_web_md')

In [30]:
comments_df = read_csv("../data/comments.csv")
tokens = comments_df["comment"].map(lambda s: word_tokenize(str(s).lower()))
tokens = tokens.map(lambda list_words: [i for i in list_words if i.isalpha() and i not in stop_words])
token_freq = Series(tokens.sum()).value_counts()

In [41]:
token_df = DataFrame(token_freq, columns=["frequency"]).reset_index().rename(columns={"index": "token"})
token_df["pos"] = token_df["token"].map(lambda x: pos_tag([x])[0][1])

In [43]:
token_df.to_csv("../data/word_frequencies.csv", index=False)

1. Multi
    - What are you looking for in your cereal?
        - taste
        - flavor
        - nutrition
        - sweet
        - crunch
    - What else do you want your cereal to have?
        - marshmallow
        - honey
        - fruit

2. Single
    - Do you have any preference for the Grain?
        - oats
        - wheat
        - rice
        - corn

### Cereal to feature association

1. Sentiment Analyzer

In [3]:
senti_analyzer = SentimentIntensityAnalyzer()

def review_sentiment(review: str):
    score = senti_analyzer.polarity_scores(review)
    return score["compound"]

2. Similarity between two documents

In [4]:
def get_spacy_word_vector_similarity(text1: str, text2: str):
    text1 = nlp(text1)
    text2 = nlp(text2)
    return text1.similarity(text2)

3. Create all possible combinations of the features that may be selected by the user

In [5]:
multi_select = dict([(chr(x[0]), x[1]) for x in enumerate(["taste", "flavor", "nutrition", "sweet", "crunch"],start=65)])
single_select_1 = dict([(chr(x[0]), x[1]) for x in enumerate(["oats", "wheat", "rice", "corn"], start=70)])
single_select_2 =  dict([(chr(x[0]), x[1]) for x in enumerate(["marshmallow", "honey", "fruit"],start=74)])

In [6]:
multi_combinations = list()
for i in range(len(multi_select.keys())+1):
    multi_combinations += list(combinations(multi_select.keys(), i))

multi_combinations[:10]

[(),
 ('A',),
 ('B',),
 ('C',),
 ('D',),
 ('E',),
 ('A', 'B'),
 ('A', 'C'),
 ('A', 'D'),
 ('A', 'E')]

4. Calculate similarity for all combinations

In [7]:
final_combinations = list()

for i in single_select_1.keys():
    final_combinations += [tuple(list(tup) + [i]) for tup in multi_combinations]
for i in single_select_2.keys():
    final_combinations += [tuple(list(tup) + [i]) for tup in multi_combinations]
for i in single_select_1.keys():
    for j in single_select_2.keys():
        final_combinations += [tuple(list(tup) + [i, j]) for tup in multi_combinations]
final_combinations += multi_combinations

final_combinations = [tuple(sorted(list(tup))) for tup in final_combinations if len(tup) > 0]
len(final_combinations)

639

In [8]:
comments_df = read_csv("../data/comments.csv")
product_df = read_csv("../data/product_rating.csv").rename(columns={"rating":"rank"})
reviews_df = comments_df.merge(product_df, on="product_id")
reviews_df["comment"] = reviews_df["comment"].map(str)
reviews_df.head()

,user,rating,comment,product_id,rank,name
0,Dino Bravo,5,Still remember the taste of it to this day. On...,4,1034,Addams Family Cereal
1,P. Crackers,5,Pretty much what everyone else is saying... th...,4,1034,Addams Family Cereal
2,Jeff H.,5,This was the,4,1034,Addams Family Cereal
3,Kaboodle Doodle,5,This was one of my favorites from the early 90's.,4,1034,Addams Family Cereal
4,K.Y.,5,I remember this cereal for some reason. I was ...,4,1034,Addams Family Cereal


In [108]:
reviews_df["sentiment_score"] = reviews_df["comment"].map(review_sentiment)
reviews_df.head()

,user,rating,comment,product_id,rank,name,sentiment_score
0,Dino Bravo,5,Still remember the taste of it to this day. On...,4,1034,Addams Family Cereal,0.6369
1,P. Crackers,5,Pretty much what everyone else is saying... th...,4,1034,Addams Family Cereal,0.9370
2,Jeff H.,5,This was the,4,1034,Addams Family Cereal,0.0000
3,Kaboodle Doodle,5,This was one of my favorites from the early 90's.,4,1034,Addams Family Cereal,0.4215
4,K.Y.,5,I remember this cereal for some reason. I was ...,4,1034,Addams Family Cereal,0.0000


In [109]:
# reviews_df.to_csv("../data/reviews.csv", index=False)

In [11]:
reviews_df = read_csv("../data/reviews.csv")
reviews_df["comment"] = reviews_df["comment"].map(str)

count = 0
for comb in final_combinations:
    col_name = "".join(map(str, sorted(list(comb))))
    if col_name+"_sim" in reviews_df.columns:
        continue
    attributes = list()
    for key in comb:
        if key in single_select_1:
            attributes.append(single_select_1[key])
        elif key in single_select_2:
            attributes.append(single_select_2[key])
        else:
            attributes.append(multi_select[key])
    attributes = " ".join(attributes)
    reviews_df[col_name+"_sim"] = reviews_df["comment"].map(lambda x: get_spacy_word_vector_similarity(x, attributes))
    reviews_df.to_csv("../data/reviews.csv", index=False)
    count+=1
    if count > 30:
        break

In [ ]:
reviews_df = read_csv("../data/reviews.csv")
reviews_df["comment"] = reviews_df["comment"].map(str)
len(reviews_df.columns)

### Cereal to Cereal association